In [182]:
import pandas as pd

In [183]:
df = pd.read_csv('trainable.csv')

In [184]:
df = df[df['metadata_type']!='percentage']
df = df[df['metadata_type']!='energy']

In [185]:
df = df[:1000]

In [186]:
df

,Unnamed: 0,image_index,orientation,text,metadata_type,metadata_value,x_min,y_min,x_max,y_max,entity_value,width,height
3,12,0,ocr_results_0,500g,weight,500g,872.0,1114.0,933.0,1144.0,500.0 gram,61.0,30.0
5,17,2,ocr_results_0,Serving Size:1 Tablet 0.709 g)Each serving con...,weight,0.709g,102.0,135.0,641.0,158.0,0.709 gram,539.0,23.0
6,20,2,ocr_results_0,200 mg,weight,200mg,725.0,223.0,783.0,254.0,0.709 gram,58.0,31.0
7,23,2,ocr_results_0,100 mg,weight,100mg,725.0,254.0,782.0,283.0,0.709 gram,57.0,29.0
10,32,2,ocr_results_0,50 mg,weight,50mg,734.0,283.0,783.0,315.0,0.709 gram,49.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,720,18,ocr_results_neg_90,31.5g,weight,31.5g,555.0,1428.0,587.0,1513.0,200 gram,32.0,85.0
197,723,18,ocr_results_neg_90,0g,weight,0g,609.0,1448.0,623.0,1508.0,200 gram,14.0,60.0
198,724,19,ocr_results_0,Regular 200gpack,weight,200g,483.0,493.0,967.0,576.0,200 gram,484.0,83.0
199,727,19,ocr_results_90,Regular 200g pack,weight,200g,436.0,1530.0,523.0,2024.0,200 gram,87.0,494.0


In [187]:
## extraction of actual text

unit_conversion = {
    'g': 1, 'gram': 1, 'grams': 1,
    'kg': 1000, 'kilogram': 1000, 'kilograms': 1000,
    'mg': 0.001, 'milligram': 0.001,
    'cm': 1, 'centimeter': 1, 'centimeters': 1,
    'm': 100, 'meter': 100, 'meters': 100,
    'inch': 2.54, 'foot': 30.48
}

def convert_to_common_unit(value, unit):
    unit = unit.lower()
    if unit in unit_conversion:
        return value * unit_conversion[unit]
    return value

In [188]:
def normalize_unit(word):
    # Convert units to a standard form
    unit_patterns = {
        r'g|gram|grams': 'gram',
        r'kg|kilogram|kilograms': 'kilogram',
        r'cm|centimetre|centimetres': 'centimetre',
        r'mm|millimetre|millimetres': 'millimetre',
        r'inch|inches': 'inch',
        r'foot|feet|ft': 'foot',
        r'lb|lbs|pound|pounds': 'pound'
        # Add more mappings if necessary
    }
    for pattern, unit in unit_patterns.items():
        if re.search(pattern, word, re.IGNORECASE):
            return unit
    return word

In [189]:
import re

def extract_number_and_unit(text):
    # Regular expression to extract number and unit
    match = re.search(r'(\d+(\.\d+)?)\s*(\w+)', text)
    if match:
        number = float(match.group(1))
        unit = match.group(3).lower()
        return number, unit
    return None, None

def normalize_value(text):
    # Normalize values by extracting number and unit
    number, unit = extract_number_and_unit(text)
    if number is not None and unit is not None:
        return convert_to_common_unit(number, unit)
    return None

In [190]:
def match_entity_value(row):
    # Extract number and unit from entity_value
    metadata_value = row['metadata_value']
    entity_value = row['entity_value']
    
    metadata_number, metadata_unit = extract_number_and_unit(metadata_value)
    entity_number, entity_unit = extract_number_and_unit(entity_value)
    
    metadata_unit = normalize_unit(metadata_unit)
    metadata_number = convert_to_common_unit(metadata_number,metadata_unit)
    
    return metadata_number == entity_number and entity_unit == metadata_unit

In [191]:
df['labels'] = df.apply(match_entity_value, axis=1)

In [192]:
df

,Unnamed: 0,image_index,orientation,text,metadata_type,metadata_value,x_min,y_min,x_max,y_max,entity_value,width,height,labels
3,12,0,ocr_results_0,500g,weight,500g,872.0,1114.0,933.0,1144.0,500.0 gram,61.0,30.0,True
5,17,2,ocr_results_0,Serving Size:1 Tablet 0.709 g)Each serving con...,weight,0.709g,102.0,135.0,641.0,158.0,0.709 gram,539.0,23.0,True
6,20,2,ocr_results_0,200 mg,weight,200mg,725.0,223.0,783.0,254.0,0.709 gram,58.0,31.0,False
7,23,2,ocr_results_0,100 mg,weight,100mg,725.0,254.0,782.0,283.0,0.709 gram,57.0,29.0,False
10,32,2,ocr_results_0,50 mg,weight,50mg,734.0,283.0,783.0,315.0,0.709 gram,49.0,32.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,720,18,ocr_results_neg_90,31.5g,weight,31.5g,555.0,1428.0,587.0,1513.0,200 gram,32.0,85.0,False
197,723,18,ocr_results_neg_90,0g,weight,0g,609.0,1448.0,623.0,1508.0,200 gram,14.0,60.0,False
198,724,19,ocr_results_0,Regular 200gpack,weight,200g,483.0,493.0,967.0,576.0,200 gram,484.0,83.0,True
199,727,19,ocr_results_90,Regular 200g pack,weight,200g,436.0,1530.0,523.0,2024.0,200 gram,87.0,494.0,True


In [193]:
##TODO: this is bad code, can make it more elegant

zero = []
plus_90 = []
minus_90 = []
for index, row in df.iterrows():
    if row['orientation'][-2] == '_':
        zero.append(1)
        plus_90.append(0)
        minus_90.append(0)
    elif row['orientation'][-4] =='g':
        zero.append(0)
        plus_90.append(0)
        minus_90.append(1)
    else:
        zero.append(0)
        plus_90.append(1)
        minus_90.append(0)

In [194]:
df['zero'] = zero
df['plus_90'] = plus_90
df['minus_90'] = minus_90

In [199]:
df

,Unnamed: 0,image_index,orientation,text,metadata_type,metadata_value,x_min,y_min,x_max,y_max,entity_value,width,height,labels,zero,plus_90,minus_90
3,12,0,ocr_results_0,500g,weight,500g,872.0,1114.0,933.0,1144.0,500.0 gram,61.0,30.0,True,1,0,0
5,17,2,ocr_results_0,Serving Size:1 Tablet 0.709 g)Each serving con...,weight,0.709g,102.0,135.0,641.0,158.0,0.709 gram,539.0,23.0,True,1,0,0
6,20,2,ocr_results_0,200 mg,weight,200mg,725.0,223.0,783.0,254.0,0.709 gram,58.0,31.0,False,1,0,0
7,23,2,ocr_results_0,100 mg,weight,100mg,725.0,254.0,782.0,283.0,0.709 gram,57.0,29.0,False,1,0,0
10,32,2,ocr_results_0,50 mg,weight,50mg,734.0,283.0,783.0,315.0,0.709 gram,49.0,32.0,False,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,720,18,ocr_results_neg_90,31.5g,weight,31.5g,555.0,1428.0,587.0,1513.0,200 gram,32.0,85.0,False,0,0,1
197,723,18,ocr_results_neg_90,0g,weight,0g,609.0,1448.0,623.0,1508.0,200 gram,14.0,60.0,False,0,0,1
198,724,19,ocr_results_0,Regular 200gpack,weight,200g,483.0,493.0,967.0,576.0,200 gram,484.0,83.0,True,1,0,0
199,727,19,ocr_results_90,Regular 200g pack,weight,200g,436.0,1530.0,523.0,2024.0,200 gram,87.0,494.0,True,0,1,0


In [200]:
w_df = df[df['metadata_type']=='weight']

In [211]:
w_df

,Unnamed: 0,image_index,orientation,text,metadata_type,metadata_value,x_min,y_min,x_max,y_max,entity_value,width,height,labels,zero,plus_90,minus_90
3,12,0,ocr_results_0,500g,weight,500g,872.0,1114.0,933.0,1144.0,500.0 gram,61.0,30.0,True,1,0,0
5,17,2,ocr_results_0,Serving Size:1 Tablet 0.709 g)Each serving con...,weight,0.709g,102.0,135.0,641.0,158.0,0.709 gram,539.0,23.0,True,1,0,0
6,20,2,ocr_results_0,200 mg,weight,200mg,725.0,223.0,783.0,254.0,0.709 gram,58.0,31.0,False,1,0,0
7,23,2,ocr_results_0,100 mg,weight,100mg,725.0,254.0,782.0,283.0,0.709 gram,57.0,29.0,False,1,0,0
10,32,2,ocr_results_0,50 mg,weight,50mg,734.0,283.0,783.0,315.0,0.709 gram,49.0,32.0,False,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,720,18,ocr_results_neg_90,31.5g,weight,31.5g,555.0,1428.0,587.0,1513.0,200 gram,32.0,85.0,False,0,0,1
197,723,18,ocr_results_neg_90,0g,weight,0g,609.0,1448.0,623.0,1508.0,200 gram,14.0,60.0,False,0,0,1
198,724,19,ocr_results_0,Regular 200gpack,weight,200g,483.0,493.0,967.0,576.0,200 gram,484.0,83.0,True,1,0,0
199,727,19,ocr_results_90,Regular 200g pack,weight,200g,436.0,1530.0,523.0,2024.0,200 gram,87.0,494.0,True,0,1,0


In [202]:
w_x = w_df[['x_min', 'y_min', 'width', 'height', 'zero', 'plus_90', 'minus_90']]
w_y = w_df['labels']

In [206]:
mean_wx_xmin = w_x['x_min'].mean()
std_wx_xmin =w_x['x_min'].std()

mean_wx_ymin = w_x['y_min'].mean()
std_wx_ymin =w_x['y_min'].std()

mean_wx_height = w_x['height'].mean()
std_wx_height =w_x['height'].std()

mean_wx_width = w_x['width'].mean()
std_wx_width =w_x['width'].std()

# Store the mean and std dev (or save them if needed)
scaling_factors = {
    'xmin': {'mean': mean_wx_xmin, 'std': std_wx_xmin},
    'ymin': {'mean': mean_wx_ymin, 'std': std_wx_ymin},
    'height': {'mean': mean_wx_height, 'std': std_wx_height},
    'width': {'mean': mean_wx_width, 'std': std_wx_width},
}

print(scaling_factors)

{'xmin': {'mean': 633.3587786259542, 'std': 387.6385919642615}, 'ymin': {'mean': 722.9465648854962, 'std': 471.48969012265303}, 'height': {'mean': 135.93893129770993, 'std': 238.01479448927574}, 'width': {'mean': 115.37404580152672, 'std': 203.82414501336353}}


In [208]:
from sklearn.preprocessing import StandardScaler

numeric_columns = ['x_min', 'y_min', 'height','width']

scaler = StandardScaler()

w_x[numeric_columns] = scaler.fit_transform(w_x[numeric_columns])

/var/folders/xx/f2vss6tn3js9s0p4_szq8w9h0000gn/T/ipykernel_18684/3834600123.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w_x[numeric_columns] = scaler.fit_transform(w_x[numeric_columns])


In [209]:
w_x

,x_min,y_min,width,height,zero,plus_90,minus_90
3,0.617991,0.832584,-0.267793,-0.446803,1,0,0
5,-1.376020,-1.251785,2.086368,-0.476325,1,0,0
6,0.237316,-1.064426,-0.282569,-0.442585,1,0,0
7,0.237316,-0.998424,-0.287494,-0.451020,1,0,0
10,0.260623,-0.936681,-0.326894,-0.438367,1,0,0
...,...,...,...,...,...,...,...
196,-0.202920,1.501114,-0.410619,-0.214837,0,0,1
197,-0.063080,1.543696,-0.499270,-0.320276,0,0,1
198,-0.389373,-0.489574,1.815492,-0.223272,1,0,0
199,-0.511085,1.718280,-0.139743,1.510140,0,1,0


In [210]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(w_x, w_y, test_size=0.2, random_state=42)

weight_model = RandomForestClassifier(random_state=42)
weight_model.fit(X_train, y_train)

y_pred = weight_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

       False       1.00      1.00      1.00        27

    accuracy                           1.00        27
   macro avg       1.00      1.00      1.00        27
weighted avg       1.00      1.00      1.00        27

